<!-- ![logog](https://raw.githubusercontent.com/Pacific-AI-Corp/langtest/refs/heads/main/docs/assets/images/logo.png) -->
![logog](https://raw.githubusercontent.com/Pacific-AI-Corp/langtest/main/docs/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Pacific-AI-Corp/langtest/blob/main/demo/tutorials/end-to-end-notebooks/Spacy_Real_World_Notebook.ipynb)

**LangTest** is an open-source python library designed to help developers deliver safe and effective Natural Language Processing (NLP) models. Whether you are using **John Snow Labs, Hugging Face, Spacy** models or **OpenAI, Cohere, AI21, Hugging Face Inference API and Azure-OpenAI** based LLMs, it has got you covered. You can test any Named Entity Recognition (NER), Text Classification, fill-mask, Translation model using the library. We also support testing LLMS for Question-Answering, Summarization and text-generation tasks on benchmark datasets. The library supports 60+ out of the box tests. For a complete list of supported test categories, please refer to the [documentation](http://langtest.org/docs/pages/docs/test_categories).

Metrics are calculated by comparing the model's extractions in the original list of sentences against the extractions carried out in the noisy list of sentences. The original annotated labels are not used at any point, we are simply comparing the model against itself in a 2 settings.

# Getting started with LangTest on spacy

In [ ]:
!pip install langtest[spacy]

# Harness and its Parameters

The Harness class is a testing class for Natural Language Processing (NLP) models. It evaluates the performance of a given NLP model on a given task using test data and generates a report with test results.Harness can be imported from the LangTest library in the following way.

In [2]:
#Import Harness from the LangTest library
from langtest import Harness

It imports the Harness class from within the module, that is designed to provide a blueprint or framework for conducting NLP testing, and that instances of the Harness class can be customized or configured for different testing scenarios or environments.

Here is a list of the different parameters that can be passed to the Harness function:

<br/>



| Parameter  | Description |  
| - | - | 
|**task**     |Task for which the model is to be evaluated (text-classification or ner)|
| **model**     | Specifies the model(s) to be evaluated. This parameter can be provided as either a dictionary or a list of dictionaries. Each dictionary should contain the following keys: <ul><li>model (mandatory): 	PipelineModel or path to a saved model or pretrained pipeline/model from hub.</li><li>hub (mandatory): Hub (library) to use in back-end for loading model from public models hub or from path</li></ul>|
| **data**      | The data to be used for evaluation. A dictionary providing flexibility and options for data sources. It should include the following keys: <ul><li>data_source (mandatory): The source of the data.</li><li>subset (optional): The subset of the data.</li><li>feature_column (optional): The column containing the features.</li><li>target_column (optional): The column containing the target labels.</li><li>split (optional): The data split to be used.</li><li>source (optional): Set to 'huggingface' when loading Hugging Face dataset.</li></ul> |
| **config**    | Configuration for the tests to be performed, specified in the form of a YAML file. |


<br/>
<br/>

## Robustness Testing

In this example, we will be testing a model's robustness. We will be applying 2 tests: uppercase and add_typo. The real-world project workflow of the model robustness testing and fixing in this case goes as follows:

1. Train NER model on original CoNLL training set

2. Test NER model robustness on CoNLL test set

3. Augment CoNLL training set based on test results 

4. Train new NER model on augmented CoNLL training set

5. Test new NER model robustness on the CoNLL test set from step 2

6. Compare robustness of new NER model against original NER model

####Load Train and Test CoNLL

In [ ]:
# Load test CoNLL
!wget https://raw.githubusercontent.com/JohnSnowLabs/langtest/main/langtest/data/conll/sample.conll

# Load train CoNLL
!wget https://raw.githubusercontent.com/JohnSnowLabs/langtest/main/demo/data/conll03.conll

#### Convert CoNLL into spaCy's JSON format:

In [4]:
import json
import spacy
from pathlib import Path

def read_conll_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read().strip()

    sentences = content.split("\n\n")

    data = []
    for sentence in sentences:
        tokens = sentence.split("\n")
        token_data = []

        for token in tokens:
            parts = token.split()
            if len(parts) == 4:
                token_data.append((parts[0], parts[3]))

        data.append(token_data)

    return data

def convert_to_spacy_format(data):
    spacy_data = []

    for tokens in data:
        text = " ".join([token[0] for token in tokens])
        entities = []

        i = 0
        while i < len(tokens):
            token, label = tokens[i]

            if label.startswith("B-"):
                start = i
                end = i
                label = label[2:]

                i += 1
                while i < len(tokens) and tokens[i][1] == f"I-{label}":
                    end = i
                    i += 1

                start_char = len(" ".join([token[0] for token in tokens[:start]])) + (1 if start > 0 else 0)
                end_char = len(" ".join([token[0] for token in tokens[:end+1]]))

                entities.append((start_char, end_char, label))
            else:
                i += 1

        spacy_data.append({"text": text, "entities": entities})

    return spacy_data

def convert_conll_to_spacy(conll_file_path, output_file_path):
    conll_data = read_conll_file(conll_file_path)
    spacy_data = convert_to_spacy_format(conll_data)

    with open(output_file_path, "w", encoding="utf-8") as file:
        json.dump(spacy_data, file, indent=2, ensure_ascii=False)



#### Provide path to train CoNLL and output path to save spaCy's JSON format

In [5]:
conll_file_path = "/content/conll03.conll"
output_file_path = "output_spacy.json"
convert_conll_to_spacy(conll_file_path, output_file_path)

#### Training Pipeline

In [6]:
import spacy
import random
import json
import os
from spacy.training import Example
from spacy.util import minibatch, compounding
from pathlib import Path

def load_data(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)
    return [(item["text"], {"entities": item["entities"]}) for item in data]

def train_ner_model(training_data, output_dir, n_iter=15):
    nlp = spacy.blank("en")
    nlp.add_pipe("ner")
    ner = nlp.get_pipe("ner")

    for _, annotations in training_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

    # Set the random seeds manually
    random.seed(1)
    os.environ["PYTHONHASHSEED"] = str(1)
    
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(training_data)
            losses = {}
            batches = minibatch(training_data, size=compounding(4.0, 32.0, 1.001))

            for batch in batches:
                examples = []
                for text, annotations in batch:
                    doc = nlp.make_doc(text)
                    example = Example.from_dict(doc, annotations)
                    examples.append(example)
                nlp.update(examples, sgd=optimizer, drop=0.35, losses=losses)

            print(f"Losses at iteration {itn}: {losses}")

    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print(f"Saved model to {output_dir}")

def load_model(model_dir):
    model_path = Path(model_dir)
    nlp = spacy.load(model_path)
    return nlp

#### Provide path to obtained spaCy's JSON format and path to save the trained model.

In [7]:
data_file = "output_spacy.json"
output_dir = "output_model"
train_data = load_data(data_file)
train_ner_model(train_data, output_dir)


Losses at iteration 0: {'ner': 6381.857948643968}
Losses at iteration 1: {'ner': 3864.764122574894}
Losses at iteration 2: {'ner': 3051.151795866062}
Losses at iteration 3: {'ner': 2499.3476493165126}
Losses at iteration 4: {'ner': 2117.314489854248}
Losses at iteration 5: {'ner': 1889.0129265490214}
Losses at iteration 6: {'ner': 1611.2352991810694}
Losses at iteration 7: {'ner': 1523.9908579266075}
Losses at iteration 8: {'ner': 1375.6149325422787}
Losses at iteration 9: {'ner': 1208.7780095227226}
Losses at iteration 10: {'ner': 1138.7647214479105}
Losses at iteration 11: {'ner': 1112.2123309850094}
Losses at iteration 12: {'ner': 926.2312691842374}
Losses at iteration 13: {'ner': 860.0685264074192}
Losses at iteration 14: {'ner': 843.776615387614}
Saved model to output_model


#### Test NER Model Robustness 

In [9]:
h = Harness(task="ner",model={"model": "output_model", "hub": "spacy"}, data={"data_source": "/content/sample.conll"})

In [10]:
h.configure({
    'tests': {
        'defaults': {'min_pass_rate': 0.60},
        
        'robustness': {
            'add_typo': {'min_pass_rate': 0.60}, 
            'uppercase':{'min_pass_rate': 0.60},
        }
    }
})

{'tests': {'defaults': {'min_pass_rate': 0.6},
 'robustness': {'add_typo': {'min_pass_rate': 0.6},
   'uppercase': {'min_pass_rate': 0.6}}}}

Here we have configured the harness to perform two robustness tests (add_typo and uppercase) and defined the minimum pass rate for each test.


#### Generating the test cases.

In [11]:
h.generate()

Generating testcases... (robustness): 100%|██████████| 1/1 [00:01<00:00,  1.59s/it]


harness.generate() method automatically generates the test cases (based on the provided configuration)

In [12]:
h.testcases()

,category,test_type,original,test_case,expected_result
0,robustness,add_typo,"SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRI...","SOCCER - JAPAN GET LUCKY WIN , CHINA IY SURPRI...","JAPAN: LOC, CHINA: LOC, SURPRISE: LOC"
1,robustness,add_typo,Nadim Ladki,Nadim aLdki,Nadim Ladki: PER
2,robustness,add_typo,"AL-AIN , United Arab Emirates 1996-12-06","AL-AIN , United Arag Emirates 1996-12-06","AL-AIN: LOC, United Arab Emirates 1996-12: ORG"
3,robustness,add_typo,Japan began the defence of their Asian Cup tit...,Japan began the defence of their Asian Cup tit...,"Japan: LOC, Asian Cup: MISC, Syria: LOC, Group..."
4,robustness,add_typo,But China saw their luck desert them in the se...,But China saw their luck desert them in the se...,"China: LOC, Uzbekistan: LOC"
...,...,...,...,...,...
447,robustness,uppercase,Portuguesa 1 Atletico Mineiro 0,PORTUGUESA 1 ATLETICO MINEIRO 0,"Portuguesa: ORG, Atletico Mineiro: PER"
448,robustness,uppercase,CRICKET - LARA ENDURES ANOTHER MISERABLE DAY .,CRICKET - LARA ENDURES ANOTHER MISERABLE DAY .,"LARA: PER, MISERABLE: MISC"
449,robustness,uppercase,Robert Galvin,ROBERT GALVIN,Robert Galvin: PER
450,robustness,uppercase,MELBOURNE 1996-12-06,MELBOURNE 1996-12-06,MELBOURNE: LOC


harness.testcases() method gives the produced test cases in form of a pandas data frame.

#### Saving test configurations, data, test cases

In [13]:
h.save("saved_test_configurations")

#### Running the tests

In [14]:
h.run()

Running test cases...: 100%|██████████| 452/452 [00:03<00:00, 125.03it/s]


Called after harness.generate() and is to used to run all the tests.  Returns a pass/fail flag for each test.

In [15]:
h.generated_results()

,category,test_type,original,test_case,expected_result,actual_result,pass
0,robustness,add_typo,"SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRI...","SOCCER - JAPAN GET LUCKY WIN , CHINA IY SURPRI...","JAPAN: LOC, CHINA: LOC, SURPRISE: LOC",JAPAN: LOC,False
1,robustness,add_typo,Nadim Ladki,Nadim aLdki,Nadim Ladki: PER,Nadim: PER,False
2,robustness,add_typo,"AL-AIN , United Arab Emirates 1996-12-06","AL-AIN , United Arag Emirates 1996-12-06","AL-AIN: LOC, United Arab Emirates 1996-12: ORG","AL-AIN: LOC, United Arag Emirates: ORG",False
3,robustness,add_typo,Japan began the defence of their Asian Cup tit...,Japan began the defence of their Asian Cup tit...,"Japan: LOC, Asian Cup: MISC, Syria: LOC, Group...","Japan: LOC, Asian Cup: MISC, Syria: LOC, Group...",True
4,robustness,add_typo,But China saw their luck desert them in the se...,But China saw their luck desert them in the se...,"China: LOC, Uzbekistan: LOC","China: LOC, Uzbekistan: LOC",True
...,...,...,...,...,...,...,...
447,robustness,uppercase,Portuguesa 1 Atletico Mineiro 0,PORTUGUESA 1 ATLETICO MINEIRO 0,"Portuguesa: ORG, Atletico Mineiro: PER","PORTUGUESA: ORG, ATLETICO MINEIRO: ORG",False
448,robustness,uppercase,CRICKET - LARA ENDURES ANOTHER MISERABLE DAY .,CRICKET - LARA ENDURES ANOTHER MISERABLE DAY .,"LARA: PER, MISERABLE: MISC","LARA: PER, MISERABLE: MISC",True
449,robustness,uppercase,Robert Galvin,ROBERT GALVIN,Robert Galvin: PER,ROBERT: MISC,False
450,robustness,uppercase,MELBOURNE 1996-12-06,MELBOURNE 1996-12-06,MELBOURNE: LOC,MELBOURNE: LOC,True


This method returns the generated results in the form of a pandas dataframe, which provides a convenient and easy-to-use format for working with the test results. You can use this method to quickly identify the test cases that failed and to determine where fixes are needed.

#### Report of the tests

In [16]:
h.report()

,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,robustness,add_typo,53,173,77%,60%,True
1,robustness,uppercase,151,75,33%,60%,False


It summarizes the results giving information about pass and fail counts and overall test pass/fail flag.

#### Augment CoNLL Training Set Based on Robustness Test Results

In [17]:
data_kwargs = {
      "data_source" : "conll03.conll",
       }

h.augment(training_data=data_kwargs, save_data_path="augmented.conll", export_mode="add")

#### Provide path to Augmented CoNLL and output path to save spaCy's JSON format for augmented conll.

In [18]:
conll_file_path = "augmented.conll"
output_file_path = "augmented_output_spacy.json"
convert_conll_to_spacy(conll_file_path, output_file_path)

#### Provide path to augmented spaCy's JSON format and path to save new augmented model.

In [19]:
data_file = "augmented_output_spacy.json"
output_dir = "augmented_output_model"
train_data = load_data(data_file)
train_ner_model(train_data, output_dir)

Losses at iteration 0: {'ner': 9605.107478641221}
Losses at iteration 1: {'ner': 6125.186236990126}
Losses at iteration 2: {'ner': 4688.077327161083}
Losses at iteration 3: {'ner': 3704.1383706821543}
Losses at iteration 4: {'ner': 3190.90800643878}
Losses at iteration 5: {'ner': 2742.468519395338}
Losses at iteration 6: {'ner': 2401.4555582425937}
Losses at iteration 7: {'ner': 2305.815699847071}
Losses at iteration 8: {'ner': 1942.8317778558853}
Losses at iteration 9: {'ner': 1858.8346100515546}
Losses at iteration 10: {'ner': 1756.4482908866023}
Losses at iteration 11: {'ner': 1732.2546920814216}
Losses at iteration 12: {'ner': 1563.8353816914448}
Losses at iteration 13: {'ner': 1464.5912482381007}
Losses at iteration 14: {'ner': 1409.599310790677}
Saved model to augmented_output_model


#### Load saved test configurations, data

In [21]:
harness = Harness.load("saved_test_configurations",
                       model={"model": "augmented_output_model","hub":"spacy"},
                       task="ner",                                 
                       load_testcases=True)

Generating testcases... (robustness): 100%|██████████| 1/1 [00:02<00:00,  2.03s/it]


#### Test New NER Model Robustness

In [22]:
harness.run().report()

Running test cases...: 100%|██████████| 452/452 [00:04<00:00, 97.65it/s] 


,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,robustness,add_typo,46,180,80%,60%,True
1,robustness,uppercase,82,144,64%,60%,True



We can see that after performing augmentation, even the **uppercase** test is passing which failed earlier.